<a href="https://colab.research.google.com/github/llorenzo62/Use_Moodle/blob/master/listados_competencias_BD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Executar** as seguintes celas

In [1]:
import re,zipfile,glob,os

def unzipODF(pathtofile):
    fzip=zipfile.ZipFile(pathtofile)
    return repr(fzip.read('content.xml').decode())



In [2]:
def extrae_tablas(direcc,min_row=2,max_col=4):
    hc=unzipODF(direcc)
    tablas=re.findall('<table:table (table:name.*?)</table:table>',hc)
    dtabla={}
    for tabla in tablas:
        nome=re.findall('table:name="(.*?)"',tabla)[0]
        if nome in ['NVScriptsProperties','DO NOT DELETE - AutoCrat Job Se']:
            continue
        #print(nome)
        rows=re.findall('<table:table-row (.*?)</table:table-row>',tabla)
        if rows:
            dtabla[nome]=[re.findall(r'<text:p>(.*?)</text:p>',item)[:max_col] for item in rows[min_row:] if item]
            dtabla[nome]=[item for item in dtabla[nome] if item]
    return dtabla

In [3]:
#carga plantilla
def plantilla():
    PP='Plantilla_BD/preset.xml'
    
    return open(PP,'r').read()

In [4]:
asignaturas='''1º de ESO:[Lingua Galega e Literatura, 
Xeografía e Historia, 
Primeira Lingua Estranxeira, 
Matemáticas, 
Segunda Lingua Estranxeira, 
Valores éticos, 
Bioloxía e Xeoloxía, 
Educación Plástica, Visual e Audiovisual, 
Educación Física, 
Lingua Castelá e Literatura], 
2º de ESO:[Lingua Galega e Literatura, 
Primeira Lingua Estranxeira, 
Xeografía e Historia, 
Matemáticas, 
Física e Química, 
Segunda Lingua Estranxeira, 
Tecnoloxía, 
Valores éticos, 
Lingua Castelá e Literatura, 
Educación Física, 
Música], 
3º de ESO:[Primeira Lingua Estranxeira, 
Xeografía e Historia, 
Segunda Lingua Estranxeira, 
Matemáticas Orientadas ás Ensinanzas Académicas, 
Bioloxía e Xeoloxía, 
Matemáticas Orientadas ás Ensinanzas Aplicadas, 
Física e Química, 
Tecnoloxía, 
Lingua Castelá e Literatura, 
Música, 
Educación Física, 
Educación Plástica, Visual e Audiovisual, 
Valores éticos, 
Cultura Clásica], 
4º de ESO:[Cultura Clásica, 
Lingua Galega e Literatura, 
Xeografía e Historia, 
Primeira Lingua Estranxeira, 
Física e Química, 
Latín, 
Lingua Castelá e Literatura, 
Filosofía, 
Segunda Lingua Estranxeira, 
Matemáticas Orientadas ás Ensinanzas Académicas, 
Tecnoloxías da Información e da Comunicación, 
Bioloxía e Xeoloxía, 
Economía, 
Matemáticas Orientadas ás Ensinanzas Aplicadas, 
Ciencias Aplicadas á Actividade Profesional, 
Tecnoloxía, 
Iniciación á Actividade Emprendedora e Empresarial, 
Educación Plástica, Visual e Audiovisual, 
Música, 
Cultura Científica, 
Educación Física, 
Valores éticos, 
Artes Escénicas e Danza] '''

listados={}
for item in asignaturas.split('],'):
    key,lst=item.split(':[')
    listados[key.strip(' \n')]=lst.strip(']').split(', \n')
listados

{'1º de ESO': ['Lingua Galega e Literatura',
  'Xeografía e Historia',
  'Primeira Lingua Estranxeira',
  'Matemáticas',
  'Segunda Lingua Estranxeira',
  'Valores éticos',
  'Bioloxía e Xeoloxía',
  'Educación Plástica, Visual e Audiovisual',
  'Educación Física',
  'Lingua Castelá e Literatura'],
 '2º de ESO': ['Lingua Galega e Literatura',
  'Primeira Lingua Estranxeira',
  'Xeografía e Historia',
  'Matemáticas',
  'Física e Química',
  'Segunda Lingua Estranxeira',
  'Tecnoloxía',
  'Valores éticos',
  'Lingua Castelá e Literatura',
  'Educación Física',
  'Música'],
 '3º de ESO': ['Primeira Lingua Estranxeira',
  'Xeografía e Historia',
  'Segunda Lingua Estranxeira',
  'Matemáticas Orientadas ás Ensinanzas Académicas',
  'Bioloxía e Xeoloxía',
  'Matemáticas Orientadas ás Ensinanzas Aplicadas',
  'Física e Química',
  'Tecnoloxía',
  'Lingua Castelá e Literatura',
  'Música',
  'Educación Física',
  'Educación Plástica, Visual e Audiovisual',
  'Valores éticos',
  'Cultura Clási

**Subir** o arquivo *Plantilla_BD.zip* e executar as seguintes celas

In [ ]:
!unzip Plantilla_BD.zip

In [ ]:
plantilla()

In [ ]:
PATH='./'


In [ ]:

f=glob.glob(PATH+'*.ods')
print(f)

In [ ]:
dictinfo={}
for item in f:

    label=item.strip().split('/')[-1].strip('.ods')
    klabel=label.split('º')
    for key,val in extrae_tablas(item).items():
        if klabel[0] in key and klabel[1] in key:
            dictinfo[label]=[item[1].title() for item in val]
            break
    for key,val in listados.items():
        if klabel[0] in key:
            dictinfo[label]=[dictinfo[label],val]
dictinfo

In [ ]:

PATH='./'
for key,val in dictinfo.items():
    alumns='\n'.join([item.title() for item in val[0]])
    asigns='\n'.join(val[1])
    preset=plantilla().format(alumns,asigns)
    os.system('mkdir "{}"'.format(PATH+key))
    os.system('cp {}Plantilla_BD/*  "{}"'.format(PATH,PATH+key))
    with open(PATH+key+'/preset.xml','w') as fich:
        fich.write(plantilla().format(alumns,asigns))
    os.system('cd "{}";zip -r "../{}.zip" * '.format(PATH+key,key))
    os.system('rm -fr "{}"'.format(PATH+key))


In [ ]:
!rm *.ods
!rm 'Plantilla_BD.zip'

In [ ]:
!zip -m listados.zip *.zip